In [1]:
import numpy as np
import pandas as pd
import datetime
pd.set_option('display.max_rows',None)

### 数据处理

In [2]:
#导入body这个字段来进行情感分析
data = pd.read_csv("./paddle.csv",usecols=["type","repo_name","created_at","body"])

/var/folders/v5/vrnscfp50517qvkjzr6km82m0000gn/T/ipykernel_70851/26559780.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./paddle.csv",usecols=["type","repo_name","created_at","body"])


In [3]:
#只要Issue评论和PR评论
data = data[data["type"].isin(["IssueCommentEvent","PullRequestReviewCommentEvent"])]
data

,type,repo_name,created_at,body
331,IssueCommentEvent,PaddlePaddle/Anakin,2018-12-03 02:14:51,can you provide the model name?
332,IssueCommentEvent,PaddlePaddle/Anakin,2018-12-03 02:17:24,是的，请用最新的master分支进行重新转换
333,IssueCommentEvent,PaddlePaddle/Anakin,2018-12-03 03:54:23,看起来还是0.1.1版本呢
334,IssueCommentEvent,PaddlePaddle/Anakin,2018-12-03 04:40:47,看到master 分支更新了。。。
335,IssueCommentEvent,PaddlePaddle/Anakin,2018-12-03 07:48:46,our model likes resnet18. it only has conv and...
...,...,...,...,...
801701,PullRequestReviewCommentEvent,PaddlePaddle/paddle-mobile,2018-11-27 07:22:03,这里是为了减少一个set接口，因为winograd需要对filter进行转换，转换后的fil...
801702,PullRequestReviewCommentEvent,PaddlePaddle/paddle-mobile,2018-11-29 00:44:14,改成fusion_dequant_add_relu?
801703,PullRequestReviewCommentEvent,PaddlePaddle/paddle-mobile,2018-11-29 00:45:30,relu的输出是Out，不是Output，Output可能会出问题
801704,PullRequestReviewCommentEvent,PaddlePaddle/paddle-mobile,2018-11-29 00:51:36,最好别在头文件中使用using吧


In [4]:
#转化为日期类型
data["created_at"] = pd.to_datetime(data["created_at"])

In [5]:
#删除内容为空的数据
data=data.dropna(axis=0,how='any')
data.reset_index(drop=True,inplace=True)

In [6]:
#提取年和月
data['year'] = data['created_at'].dt.strftime("%Y")
data['month'] = data['created_at'].dt.strftime("%m")
data.drop(columns=['created_at'],inplace=True)

In [8]:
data.to_csv("评论.csv",index=False)

### 方法1:利用paddle的senta模型

In [ ]:
#!hub install senta_bilstm

In [14]:
#调用senta预训练模型对评论进行情感分析
import os
import gc
import paddlehub as hub
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
senta =hub.Module(name="senta_bilstm")
#CUDA_VISIBLE_DEVICES as cuda_device_id



[2022-09-19 13:03:46,682] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [16]:
# del res
# gc.collect()

5509

In [17]:
text = data['body'].to_list()
res = senta.sentiment_classify(texts=text, 
                                use_gpu=True,
                                batch_size=1000)

[2022-09-19 13:04:25,463] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [18]:
len(text)

276896

In [21]:
res[0]

{'text': 'can you provide the model name?',
 'sentiment_label': 0,
 'sentiment_key': 'negative',
 'positive_probs': 0.4386,
 'negative_probs': 0.5614}

In [23]:
flag = []
for i in range(len(res)):
    flag.append(res[i]["sentiment_label"])
flag = pd.DataFrame(flag)
flag.rename(columns={0:'flag'})
data = pd.concat([data,flag],axis=1)

In [43]:
data

,type,body,year,month,flag
0,IssueCommentEvent,can you provide the model name?,2018,12,0
1,IssueCommentEvent,是的，请用最新的master分支进行重新转换,2018,12,1
2,IssueCommentEvent,看起来还是0.1.1版本呢,2018,12,0
3,IssueCommentEvent,看到master 分支更新了。。。,2018,12,0
4,IssueCommentEvent,our model likes resnet18. it only has conv and...,2018,12,1
...,...,...,...,...,...
276891,PullRequestReviewCommentEvent,这里是为了减少一个set接口，因为winograd需要对filter进行转换，转换后的fil...,2018,11,1
276892,PullRequestReviewCommentEvent,改成fusion_dequant_add_relu?,2018,11,0
276893,PullRequestReviewCommentEvent,relu的输出是Out，不是Output，Output可能会出问题,2018,11,0
276894,PullRequestReviewCommentEvent,最好别在头文件中使用using吧,2018,11,0


### 方法2:利用腾讯云API

In [ ]:
def SentimentAnalysis(text):
    try:
        # 实例化一个认证对象，入参需要传入腾讯云账户secretId，secretKey,此处还需注意密钥对的保密
        # 密钥可前往https://console.cloud.tencent.com/cam/capi网站进行获取
        cred = credential.Credential("secretId", "secretKey")
        # 实例化一个http选项，可选的，没有特殊需求可以跳过
        httpProfile = HttpProfile()
        httpProfile.endpoint = "nlp.tencentcloudapi.com"

        # 实例化一个client选项，可选的，没有特殊需求可以跳过
        clientProfile = ClientProfile()
        clientProfile.httpProfile = httpProfile
        # 实例化要请求产品的client对象,clientProfile是可选的
        client = nlp_client.NlpClient(cred, "ap-guangzhou", clientProfile)

        # 实例化一个请求对象,每个接口都会对应一个request对象
        req = models.SentimentAnalysisRequest()
        params = {
            "Text": text,
            "Flag": 4,
            "Mode": "3class"
        }
        req.from_json_string(json.dumps(params))

        # 返回的resp是一个SentimentAnalysisResponse的实例，与请求对象对应
        resp = client.SentimentAnalysis(req)
        # 输出json格式的字符串回包
        print(resp.to_json_string())

    except TencentCloudSDKException as err:
        print(err)

In [2]:
#导入情感分析结果数据
data = pd.read_csv("情感分析结果.csv")


In [3]:
data['flag'] = data['flag'].fillna('neutral')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276896 entries, 0 to 276895
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   type       276896 non-null  object
 1   body       276896 non-null  object
 2   year       276896 non-null  int64 
 3   month      276896 non-null  int64 
 4   flag       276896 non-null  object
 5   repo_name  276896 non-null  object
dtypes: int64(2), object(4)
memory usage: 12.7+ MB


### 整体分析

In [7]:
#情感分析（整体）
data['flag'].value_counts()

positive    135680
neutral      97058
negative     44158
Name: flag, dtype: int64

In [9]:
#情感分析（年）
#data.groupby(data['year'])['flag'].value_counts()
data.groupby(data['year'])['flag'].value_counts(normalize=True)

year  flag    
2016  positive    0.452935
      neutral     0.390181
      negative    0.156884
2017  positive    0.593854
      neutral     0.295982
      negative    0.110164
2018  positive    0.584862
      neutral     0.296794
      negative    0.118344
2019  positive    0.500334
      neutral     0.353029
      negative    0.146637
2020  positive    0.492758
      neutral     0.360715
      negative    0.146526
2021  positive    0.461087
      neutral     0.349823
      negative    0.189089
2022  positive    0.414371
      neutral     0.398734
      negative    0.186895
Name: flag, dtype: float64

In [10]:
#情感分析（月）
#data.groupby([data['year'],data['month']])['flag'].value_counts()
data.groupby([data['year'],data['month']])['flag'].value_counts(normalize=True)

year  month  flag    
2016  11     neutral     0.439601
             positive    0.414695
             negative    0.145704
      12     positive    0.469045
             neutral     0.369360
             negative    0.161595
2017  1      neutral     0.422954
             positive    0.409139
             negative    0.167906
      2      positive    0.512090
             neutral     0.333333
             negative    0.154577
      3      positive    0.527300
             neutral     0.314884
             negative    0.157816
      4      positive    0.534038
             neutral     0.334507
             negative    0.131455
      5      positive    0.512234
             neutral     0.385479
             negative    0.102286
      6      positive    0.626439
             neutral     0.288897
             negative    0.084664
      7      positive    0.621849
             neutral     0.275849
             negative    0.102302
      8      positive    0.642047
             neutral     0

### Issue评论分析

In [12]:
df1 = data[data["type"].isin(["IssueCommentEvent"])]

In [13]:
#情感分析（整体）
df1['flag'].value_counts()

positive    69498
neutral     67598
negative    28514
Name: flag, dtype: int64

In [14]:
#情感分析（年）
#df1.groupby(df1['year'])['flag'].value_counts()
df1.groupby(df1['year'])['flag'].value_counts(normalize=True)

year  flag    
2016  neutral     0.431737
      positive    0.425304
      negative    0.142959
2017  positive    0.500000
      neutral     0.356374
      negative    0.143626
2018  positive    0.468770
      neutral     0.385232
      negative    0.145999
2019  neutral     0.440715
      positive    0.394291
      negative    0.164994
2020  positive    0.436380
      neutral     0.420098
      negative    0.143522
2021  positive    0.420738
      neutral     0.381666
      negative    0.197596
2022  neutral     0.448523
      positive    0.361252
      negative    0.190225
Name: flag, dtype: float64

In [15]:
#情感分析（月）
#df1.groupby([df1['year'],df1['month']])['flag'].value_counts()
df1.groupby([df1['year'],df1['month']])['flag'].value_counts(normalize=True)

year  month  flag    
2016  11     neutral     0.479545
             positive    0.393182
             negative    0.127273
      12     positive    0.440042
             neutral     0.409802
             negative    0.150156
2017  1      positive    0.476460
             neutral     0.365348
             negative    0.158192
      2      positive    0.453039
             neutral     0.377532
             negative    0.169429
      3      positive    0.496703
             neutral     0.329670
             negative    0.173626
      4      positive    0.532468
             neutral     0.347403
             negative    0.120130
      5      positive    0.474870
             neutral     0.415945
             negative    0.109185
      6      positive    0.457526
             neutral     0.411326
             negative    0.131148
      7      positive    0.513823
             neutral     0.329838
             negative    0.156339
      8      positive    0.529786
             neutral     0

### PR评论分析

In [16]:
df2 = data[data["type"].isin(["PullRequestReviewCommentEvent"])]

In [17]:
#情感分析（整体）
df2['flag'].value_counts()

positive    66182
neutral     29460
negative    15644
Name: flag, dtype: int64

In [20]:
#情感分析（年）
df2.groupby(df2['year'])['flag'].value_counts(normalize=True)

year  flag    
2016  positive    0.482443
      neutral     0.345802
      negative    0.171756
2017  positive    0.649175
      neutral     0.260384
      negative    0.090440
2018  positive    0.706198
      neutral     0.204362
      negative    0.089440
2019  positive    0.630698
      neutral     0.245232
      negative    0.124069
2020  positive    0.596067
      neutral     0.251901
      negative    0.152032
2021  positive    0.541338
      neutral     0.286492
      negative    0.172170
2022  positive    0.510546
      neutral     0.308590
      negative    0.180865
Name: flag, dtype: float64

In [21]:
#情感分析（月）
df2.groupby([df2['year'],df2['month']])['flag'].value_counts(normalize=True)

year  month  flag    
2016  11     positive    0.440771
             neutral     0.391185
             negative    0.168044
      12     positive    0.498416
             neutral     0.328405
             negative    0.173178
2017  1      neutral     0.445596
             positive    0.382679
             negative    0.171725
      2      positive    0.564228
             neutral     0.294309
             negative    0.141463
      3      positive    0.543084
             neutral     0.307256
             negative    0.149660
      4      positive    0.534926
             neutral     0.327206
             negative    0.137868
      5      positive    0.523486
             neutral     0.376305
             negative    0.100209
      6      positive    0.682493
             neutral     0.248269
             negative    0.069238
      7      positive    0.688981
             neutral     0.242299
             negative    0.068720
      8      positive    0.727603
             neutral     0

### 仓库分析

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276896 entries, 0 to 276895
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   type       276896 non-null  object
 1   body       276896 non-null  object
 2   year       276896 non-null  int64 
 3   month      276896 non-null  int64 
 4   flag       276896 non-null  object
 5   repo_name  276896 non-null  object
dtypes: int64(2), object(4)
memory usage: 12.7+ MB


In [6]:
data = data[data['repo_name'].isin(["PaddlePaddle/Paddle","PaddlePaddle/PaddleOCR","PaddlePaddle/PaddleNLP","PaddlePaddle/PaddleDetection","PaddlePaddle/PaddleSpeech"])]

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183356 entries, 64 to 276792
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   type       183356 non-null  object
 1   body       183356 non-null  object
 2   year       183356 non-null  int64 
 3   month      183356 non-null  int64 
 4   flag       183356 non-null  object
 5   repo_name  183356 non-null  object
dtypes: int64(2), object(4)
memory usage: 9.8+ MB
